In [1]:
from visiprog.data import *
from visiprog.metriclearning import *
from visiprog.evaluate import *
%matplotlib inline
import numpy as np
from visiprog.metric_learn.fda_visiprog import LFDA_VISIPROG

from IPython.display import Image, display
from sklearn.model_selection import train_test_split


In [2]:
raw_feature = np.genfromtxt('visiprog/data/curetaliasfree.csv',delimiter=",")
label_material = read_material_label()
groups_VSP, N = read_VSP_label()

print(raw_feature.shape)
print(label_material.shape)

(5245, 82)
(5245,)


In [4]:
label_visiprog = np.zeros((N,))

count = 0
for g in groups_VSP:
    label = False
    
    # check whether any entry in the group has been labelled yet
    # if yes, then merge with that existing label
    for i in g:
        if label_visiprog[i] != 0:
            label = label_visiprog[i]
            break
    
    # if no, then create a new label
    if label == False:
        count += 1
        label = count
        
    for i in g:
        label_visiprog[i] = label


In [5]:
accuracy = []

for i in range(10):
    X_train, X_test, Y_mat_train, Y_mat_test, Y_VSP_train, Y_VSP_test = \
    train_test_split(raw_feature, label_material, label_visiprog, \
                     test_size=0.5, stratify=label_material)


    model = LFDA_VISIPROG()
    model.fit(X_train, Y_VSP_train)

    X_vsp_test = model.transform(X_test)

    res = leave_one_sample_out(X_vsp_test, Y_mat_test)
    accuracy.append(res['accuracy'])

In [6]:
display(accuracy)

[0.9729317575295463,
 0.96950057186427752,
 0.97826915745329779,
 0.9729317575295463,
 0.97560045749142199,
 0.97750667174990469,
 0.97216927182615331,
 0.98017537171178037,
 0.97140678612276021,
 0.97941288600838738]

In [7]:
np.array(accuracy).mean()

0.97499046892870744